<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/03_transformer_encoder_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Transformer encoder for text classification

Factoring outputs into multiple independent spaces, adding residual connections, adding normalization layers—all of these are standard architecture patterns that one would be wise to leverage in any complex model. Together, these bells and whistles form the Transformer encoder—one
of two critical parts that make up the Transformer architecture.

<img src='https://github.com/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/images/2.png?raw=1' width='600'/>

The original Transformer architecture consists of two parts: a Transformer encoder that processes the source sequence, and a Transformer decoder that uses the source sequence to generate a translated version.

Crucially, the encoder part can be used for text classification—it’s a very generic module that ingests a sequence and learns to turn it into a more useful representation.

Let’s implement a Transformer encoder and try it on the movie review sentiment
classification task.





##Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import os, pathlib, shutil, random

Let’s start by downloading the dataset from the Stanford page and uncompressing it:

In [ ]:
%%shell

curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xf aclImdb_v1.tar.gz

# delete unwanted file and subdirectory
rm -rf aclImdb/train/unsup
rm -rf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  16.7M      0  0:00:04  0:00:04 --:--:-- 17.9M


In [ ]:
# take a look at the content of a few of these text files
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

let’s download the GloVe word embeddings.

In [ ]:
%%shell

wget http://nlp.stanford.edu/data/glove.6B.zip
unzip -q glove.6B.zip

##Preparing the IMDB movie reviews data

For instance, the `train/pos/` directory contains a set of `12,500` text files, each of which
contains the text body of a positive-sentiment movie review to be used as training data.
The negative-sentiment reviews live in the “neg” directories. 

In total, there are `25,000`
text files for training and another 25,000 for testing.

Next, let’s prepare a validation set by setting apart 20% of the training text files in a new directory, `aclImdb/val`:

In [ ]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code
  random.Random(1337).shuffle(files)
  # Take 20% of the training files to use for validation
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    # Move the files to aclImdb/val/neg and aclImdb/val/pos
    shutil.move(train_dir/category/fname, val_dir/category/fname)

Remember how, we used the `image_dataset_from_directory` utility to
create a batched Dataset of images and their labels for a directory structure? You can do the exact same thing for text files using the `text_dataset_from_directory` utility.

Let’s create three Dataset objects for training, validation, and testing:

In [ ]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


These datasets yield inputs that are TensorFlow `tf.string` tensors and targets that are `int32` tensors encoding the value “0” or “1.”

In [ ]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Not an altogether bad start for the program -- but what a slap in the face to real law enforcement. The worst part of the series is that it attempts to bill itself as reality fare -- and is anything but. Men and women that dedicate their lives to the enforcement of laws deserve better than this. What is next, medical school in a minute? Charo performing lipo? Charles Grodin assisting on a hip replacement? C'mon...show a little respect. Even the citizens of Muncie are outing the program as staged. Police Academy = High School Gym? Poor editing (how many times can they use the car-to-car shot of the Taco Bell in the background?), cheesy siren effects (the same loop added ad nauseum to every 'call' whether rolling code or not), and last, but not least -- more officer safety issues than you could shake a stick at.<br /><br />If I want to see manufactured police wo

###Preparing datasets for sequences

First, let’s prepare datasets that return integer sequences.

In [ ]:
max_length = 600
max_tokens = 20000

# Prepare a dataset that only yields raw text inputs (no labels).
text_only_train_ds = train_ds.map(lambda x, y: x)

# This is a reasonable choice, since the average review length is 233 words, and only 5% of reviews are longer than 600 words.
text_vectorization = layers.TextVectorization(max_tokens=max_tokens,
                                              output_mode="int",
                                              output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

##Transformer encoder

Let’s implement a Transformer encoder and try it on the movie review sentiment
classification task.

In [ ]:
class TransformerEncoder(layers.Layer):
  pass

In [ ]:
# Prepare a matrix that we’ll fill with the GloVe vectors.
embedding_matrix = np.zeros((max_tokens, embedding_dim))

for word, i in word_index.items():
  # Fill entry i in the matrix with the word vector for index i. Words not found in the embedding index will be all zeros.
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

Finally, we use a Constant initializer to load the pretrained embeddings in an Embedding layer. 

So as not to disrupt the pretrained representations during training, we freeze
the layer via `trainable=False`:

In [ ]:
embedding_layer = layers.Embedding(max_tokens, embedding_dim, 
                                   embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                   trainable=False, mask_zero=True)

We’re now ready to train a new model—identical to our previous model, but leveraging the `100-dimensional` pretrained GloVe embeddings instead of `128-dimensional` learned embeddings.

In [ ]:
# One input is a sequence of integers
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = embedding_layer(inputs)
# add a bidirectional LSTM
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               34048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
______________________________________________

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras", save_best_only=True)]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)

model = keras.model.load_model("glove_embeddings_sequence_model.keras")
print(f"Test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

You’ll find that on this particular task, pretrained embeddings aren’t very helpful, because the dataset contains enough samples that it is possible to learn a specialized enough embedding space from scratch. 

However, leveraging pretrained embeddings can be very helpful when you’re working with a smaller dataset.